In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader

# Define the CNN architecture
class PersonRecognitionCNN(nn.Module):
    def __init__(self):
        super(PersonRecognitionCNN, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 32 * 32, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader

# Set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set random seed for reproducibility
torch.manual_seed(42)

# Define hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Define image dimensions
image_width = 640
image_height = 480

# Load and transform the COCO dataset
transform = transforms.Compose([
    transforms.Resize((image_height, image_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = CocoDetection(root='C:/Users/lolol/OneDrive/Документы/СOCO Dataset/train2017', annFile='C:/Users/lolol/OneDrive/Документы/СOCO Dataset/annotations/instances_train2017.json', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Create the model instance
model = PersonRecognitionCNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, targets in train_loader:
        images = images.to(device)
        labels = targets['labels'].to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {running_loss / len(train_loader):.4f}")

# Save the trained model
torch.save(model.state_dict(), 'person_recognition_model.pth')


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader

# Set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set random seed for reproducibility
torch.manual_seed(42)

# Define hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Load and transform the COCO dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CocoDetection(root='C:/Users/lolol/OneDrive/Документы/СOCO Dataset/train2017', annFile='C:/Users/lolol/OneDrive/Документы/СOCO Dataset/annotations/instances_train2017.json', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Create the model instance
model = fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2  # Person or not person
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = nn.Linear(in_features, num_classes)

# Move model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, targets in train_loader:
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in target.items()} for target in targets]

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backward and optimize
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()

    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {running_loss / len(train_loader):.4f}")

# Save the trained model
torch.save(model.state_dict(), 'person_recognition_model.pth')

    
torch.save(model.state_dict(), 'person_recognition_model.pth')


loading annotations into memory...
Done (t=141.86s)
creating index...
index created!


c:\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: stack expects each tensor to be equal size, but got [3, 480, 640] at entry 0 and [3, 361, 640] at entry 1

In [ ]:
test_dataset = CocoDetection(root='path/to/test_dataset', annFile='path/to/test_annotations.json', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Testing
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, targets in test_loader:
        images = images.to(device)
        labels = targets['labels'].to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")